## LSTM and CNN results comparison

1. We will first calculate the error for each time series
2. Then we will calculate the error for each level in the hierarchy
3. Then we will calculate get the forecasts at the grid level
4. Finally we will calculate the error for the grid level

In [1]:
import sys
sys.path.insert(0, '../')

import src.utils as util
import src.calculate_errors as err
import constants as const

%load_ext autoreload
%autoreload 2

In [2]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [3]:
import pandas as pd
import numpy as np

In [4]:
def calculate_errors_per_ts(model_path, ts):
    data = pd.read_csv(f'../ts_data/{ts}.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    denom = err.calculate_denom(train_df, const.H_SEASONALITY)
    
    results_df = pd.read_csv(f'../{model_path}/final_results/{ts}.csv', index_col=[0])
    test_sample = results_df[['power']].values
    forecasts = results_df[['average_fc']].values
    horizon = 14
    
    mean_mase, error_dist = err.test_errors(train_df, test_sample, forecasts, horizon, const.H_SEASONALITY, denom)
    return mean_mase, error_dist

In [5]:
def get_results_ts(model_path):
    ts_mase = {}
    ts_dist_err = {}
    mase_vals = []

    for ts in const.TS:
        mean_mase_ts, ts_dist = calculate_errors_per_ts(model_path, ts)
        ts_dist_err[ts] = ts_dist
        ts_mase[ts] = mean_mase_ts
        mase_vals.append(mean_mase_ts)
    return ts_mase, ts_dist_err, mase_vals

In [6]:
lstm_mase_ts, lstm_dist_ts, mase_vals_lstm = get_results_ts('lstm_results')

In [7]:
# tcn_mase_ts, tcn_dist_ts, mase_vals_cnn = get_results_ts('cnn_results/tcn')

tcn_mase_ts, tcn_dist_ts, mase_vals_cnn = get_results_ts('cnn_results/tcn_new/dilation_2')

In [8]:
tcn2_mase_ts, tcn2_dist_ts, mase_vals_cnn2 = get_results_ts('cnn_results/tcn_new/filter_64')

In [18]:
lstm2_mase_ts, lstm2_dist_ts, mase_vals_lstm2 = get_results_ts('lstm_results/lstm2')

In [107]:
# tcn_d3_mase_ts, tcn_d3_dist_ts, mase_vals_cnn_d3 = get_results_ts('cnn_results/tcn/dilation_3')

In [108]:
# tcn_d1_mase_ts, tcn_d1_dist_ts, mase_vals_cnn_d1 = get_results_ts('cnn_results/tcn/dilation_1')

plot error in chart

In [19]:
fig = go.Figure(data=[
    go.Bar(name='LSTM', x=const.TS, y=mase_vals_lstm),
    go.Bar(name='LSTM-2', x=const.TS, y=mase_vals_lstm2),
#     go.Bar(name='Dilated Convolutions (Rate - 1)', x=const.TS, y=mase_vals_cnn_d1),
    go.Bar(name='Dilated Convolutions (Rate - 2)', x=const.TS, y=mase_vals_cnn),
    go.Bar(name='Dilated Convolutions (Filter - 64)', x=const.TS, y=mase_vals_cnn2)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title='Mean MASE - 36 samples (1 day ahead - 1 hour resolution)')
fig.show()

Error distributions

In [22]:
fig = go.Figure()
for col in lstm_dist_ts.keys():
    box_data = lstm_dist_ts[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'Time series')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "LSTM", showlegend= False)

In [111]:
fig = go.Figure()
for col in tcn_dist_ts.keys():
    box_data = tcn_dist_ts[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'Time series')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Convolutions", showlegend= False)

In [11]:
# fig = go.Figure()
# for col in tcn_d3_dist_ts.keys():
#     box_data = tcn_d3_dist_ts[col]
#     fig.add_trace(go.Box(y=box_data, name = col))
# fig.update_xaxes(title = 'Time series')
# fig.update_yaxes(title = 'MASE distribution')
# fig.update_layout(title = "Dilated Convolutions (Rate - 3)", showlegend= False)

## Errors at each level

In [10]:
def get_fc_each_level(start, end, mase_dict):
    if end == -1:
        ts_names = const.TS[start: ]
    else:
        ts_names = const.TS[start: end]
    mase_vals= []
    for ts in ts_names: 
        mase_vals.append(mase_dict[ts])
    avg_mase = np.mean(mase_vals)
    return avg_mase

In [11]:
def get_fc_error_level_model(model_mase_dic):
    level_mase = []
    grid = get_fc_each_level(0, 1, model_mase_dic)
    tl_line = get_fc_each_level(1, 3, model_mase_dic)
    sub = get_fc_each_level(3, 7, model_mase_dic)
    pc = get_fc_each_level(7, 13, model_mase_dic)
    site = get_fc_each_level(13, -1, model_mase_dic)
    level_mase.append(grid)
    level_mase.append(tl_line)
    level_mase.append(sub)
    level_mase.append(pc)
    level_mase.append(site)
    return level_mase

In [12]:
level_results_lstm = get_fc_error_level_model(lstm_dist_ts)

In [13]:
level_results_lstm

[1.4682344822944209,
 1.2524748102053058,
 1.0567580454460388,
 0.8120909452654899,
 0.850674400686877]

In [14]:
level_results_tcn = get_fc_error_level_model(tcn_dist_ts)

In [16]:
level_results_tcn2 = get_fc_error_level_model(tcn2_dist_ts)

In [17]:
x_vals = ['Grid', 'Transmission', 'Substation', 'PostCode', 'Site']
fig = go.Figure(data=[
    go.Bar(name='LSTM', x=x_vals, y=level_results_lstm),
    go.Bar(name='Convolutions', x=x_vals, y=level_results_tcn),
    go.Bar(name='Convolutions-F64', x=x_vals, y=level_results_tcn2)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title='Average MASE at each level in the time series hierarchy')
fig.show()

## Grid level Forecasts

In [7]:
def sum_fc_results(ts_array, model_path):
    dfs = []
    for ts in ts_array:
        ts_fc = pd.read_csv(f'../{model_path}/final_results/{ts}.csv', index_col=[0])[['average_fc']]
        dfs.append(ts_fc)
    concat_df = pd.concat(dfs, axis=1).sum(axis=1)
    return concat_df

In [8]:
def calculate_grid_error(start, end, model_path, error_metric):
    if end == -1:
        ts_array = const.TS[start: ]
    else:
        ts_array = const.TS[start: end]
    data = pd.read_csv(f'../ts_data/grid.csv', index_col=[0])
    look_back = 14 * 7  # 14 hours in to 7 days

    # train, val, test split
    train, val, test = util.split_hourly_data(data, look_back)
    train_df = train[['power']]
    denom = err.calculate_denom(train_df, const.H_SEASONALITY)
    
    results_df = pd.read_csv(f'../{model_path}/final_results/grid.csv', index_col=[0])
    test_sample = results_df['power'].values
    forecasts = sum_fc_results(ts_array, model_path).values
    horizon = 14
    
    if error_metric == "RMSE":
        mean_err, error_dist = err.test_errors_nrmse(train_df.values, test_sample, forecasts, horizon)
    else:
        mean_err, error_dist = err.test_errors(train_df, test_sample, forecasts, horizon, const.H_SEASONALITY, denom)
        
    return mean_err, error_dist

In [9]:
def mase_grid(model_path, error_metric = "MASE"):
    level_mase = []
    level_mase_dist = {}
    grid_mase, grid_dist = calculate_grid_error(0, 1, model_path, error_metric)
    tl_mase, tl_dist = calculate_grid_error(1, 3, model_path, error_metric)
    sub_mase, sub_dist = calculate_grid_error(3, 7, model_path, error_metric)
    pc_mase, pc_dist = calculate_grid_error(7, 13, model_path, error_metric)
    site_mase, site_dist = calculate_grid_error(13, -1, model_path, error_metric)
    
    level_mase.append(grid_mase)
    level_mase.append(tl_mase)
    level_mase.append(sub_mase)
    level_mase.append(pc_mase)
    level_mase.append(site_mase)
    
    
    level_mase_dist['grid'] = grid_dist
    level_mase_dist['tl'] = tl_dist
    level_mase_dist['sub'] = sub_dist
    level_mase_dist['pc'] = pc_dist
    level_mase_dist['site'] = site_dist
    
    return level_mase, level_mase_dist

In [10]:
grid_error_naive, grid_error_dist_naive = mase_grid('benchmark_results/naive')

In [11]:
grid_error_naive

[1.079391787981905,
 1.079391787981905,
 1.079391787981905,
 1.079391787981905,
 1.079391787981905]

In [12]:
grid_error_naive_rmse, grid_error_dist_naive_rmse = mase_grid('benchmark_results/naive', "RMSE")

In [13]:
grid_error_naive_rmse

[0.3965041238428431,
 0.3965041238428431,
 0.3965041238428431,
 0.39650412384284306,
 0.39650412384284306]

In [14]:
grid_error_arima, grid_error_dist_arima = mase_grid('benchmark_results/arima')

In [15]:
grid_error_arima

[1.0542174171354002,
 1.023268311933142,
 1.024157068352861,
 0.929386914423518,
 1.0084912706608253]

In [18]:
grid_error_arima_rmse, grid_error_dist_arima_rmse = mase_grid('benchmark_results/arima', "RMSE")

In [19]:
grid_error_arima_rmse

[0.3604885973574808,
 0.34673791723819,
 0.3472883436134808,
 0.31814541816116665,
 0.3358448691063564]

In [33]:
grid_error_dist_arima_rmse['grid'][14]

0.7443561393233701

In [34]:
grid_error_dist_tbats_rmse['grid'][14]

0.8806265803815145

In [21]:
grid_error_tbats, grid_error_dist_tbats = mase_grid('benchmark_results/tbats')

In [22]:
grid_error_tbats

[1.8748277728587728,
 1.7491129158069132,
 1.706391431091947,
 1.5437416506766872,
 1.683218533714503]

In [23]:
grid_error_tbats_rmse, grid_error_dist_tbats_rmse = mase_grid('benchmark_results/tbats', "RMSE")

In [24]:
grid_error_tbats_rmse

[0.568815440036228,
 0.5301626039525603,
 0.5172115824644526,
 0.4796227263138744,
 0.512242338766708]

In [38]:
grid_error_prophet, grid_error_dist_prophet = mase_grid('benchmark_results/prophet')

In [39]:
grid_error_prophet

[1.962394493488318,
 1.9665700550555816,
 1.966532036884485,
 1.0803983146720384,
 1.0899355671492128]

In [40]:
grid_error_prophet2, grid_error_dist_prophet2 = mase_grid('benchmark_results/prophet_2')

In [41]:
grid_error_prophet2

[1.9616519408739086,
 1.9663323075008177,
 1.966214992364306,
 1.076739346631657,
 1.0865213639116051]

In [42]:
grid_error_prophet_rmse, grid_error_dist_prophet_rmse = mase_grid('benchmark_results/prophet', "RMSE")

In [43]:
grid_error_prophet_rmse

[0.589810259593252,
 0.5913052140865176,
 0.591272299547771,
 0.3701723039131422,
 0.37032535308201564]

In [44]:
grid_error_prophet2_rmse, grid_error_dist_prophet2_rmse = mase_grid('benchmark_results/prophet_2', "RMSE")

In [45]:
grid_error_prophet2_rmse

[0.5894616278192185,
 0.5912752348094751,
 0.5912942988749715,
 0.37368530172319714,
 0.3737810116341059]

In [55]:
grid_error, grid_error_dist = mase_grid('lstm_results')

In [56]:
grid_error

[1.4682344822944209,
 1.4161838663439736,
 1.3301243856811835,
 1.1232270030823759,
 1.0130934871185588]

In [48]:
grid_error_rmse, grid_error_dist_rmse = mase_grid('lstm_results', "RMSE")

In [49]:
# grid_error_lstm2, grid_error_dist_lstm2 = mase_grid('dnn_results/lstm')

In [50]:
# grid_error_lstm2

In [51]:
# grid_error_tcn, grid_error_dist_tcn = mase_grid('cnn_results/tcn')

grid_error_tcn, grid_error_dist_tcn = mase_grid('cnn_results/tcn_new/dilation_2')

In [52]:
grid_error_tcn

[1.150048520316878,
 1.1694360905458727,
 1.1717213662474213,
 1.198340986831758,
 1.1695815390243385]

In [35]:
grid_error_tcn_rmse, grid_error_dist_tcn_rmse = mase_grid('cnn_results/tcn_new/dilation_2', "RMSE")

In [36]:
grid_error_tcn_rmse

[0.377380092919877,
 0.38148387426162045,
 0.38206648791517106,
 0.39047288944725306,
 0.37784276340728884]

In [57]:
grid_error_lstm2, grid_error_dist_lstm2 = mase_grid('lstm_results/lstm2')

In [58]:
grid_error_lstm2_rmse, grid_error_dist_lstm2_rmse = mase_grid('lstm_results/lstm2', "RMSE")

In [84]:
# grid_error_tcn

In [59]:
def get_df_method(error_list, method):
    df = pd.DataFrame(error_list).transpose()
    df.columns = ['Grid', 'TL - Aggregated', 'SUB - Aggregated', 'PC - Aggregated', 'Site - Aggregated']
    df.index = [method]
    return df

In [60]:
lstm = get_df_method(grid_error, "LSTM")
tcn = get_df_method(grid_error_tcn, "CNN")
arima = get_df_method(grid_error_arima, "ARIMA")
tbats = get_df_method(grid_error_tbats, "TBATS")
prophet = get_df_method(grid_error_prophet, "PROPHET")
naive = get_df_method(grid_error_naive, "NAIVE")
lstm2 = get_df_method(grid_error_lstm2, "LSTM-2")

mase_final_results = pd.concat([naive, arima, tbats, prophet, tcn, lstm, lstm2]).round(3)

In [61]:
mase_final_results

,Grid,TL - Aggregated,SUB - Aggregated,PC - Aggregated,Site - Aggregated
NAIVE,1.079,1.079,1.079,1.079,1.079
ARIMA,1.054,1.023,1.024,0.929,1.008
TBATS,1.875,1.749,1.706,1.544,1.683
PROPHET,1.962,1.967,1.967,1.080,1.090
CNN,1.150,1.169,1.172,1.198,1.170
LSTM,1.468,1.416,1.330,1.123,1.013
LSTM-2,1.406,1.352,1.383,1.151,1.104


In [96]:
mase_final_results.to_csv('mase_results.csv')

In [62]:
lstm = get_df_method(grid_error_rmse, "LSTM")
tcn = get_df_method(grid_error_tcn_rmse, "CNN")
arima = get_df_method(grid_error_arima_rmse, "ARIMA")
tbats = get_df_method(grid_error_tbats_rmse, "TBATS")
prophet = get_df_method(grid_error_prophet_rmse, "PROPHET")
naive = get_df_method(grid_error_naive_rmse, "NAIVE")
lstm2 = get_df_method(grid_error_lstm2_rmse, "LSTM-2")


nrmse_final_results = pd.concat([naive, arima, tbats, prophet, tcn, lstm, lstm2]).round(3)
nrmse_final_results

,Grid,TL - Aggregated,SUB - Aggregated,PC - Aggregated,Site - Aggregated
NAIVE,0.397,0.397,0.397,0.397,0.397
ARIMA,0.360,0.347,0.347,0.318,0.336
TBATS,0.569,0.530,0.517,0.480,0.512
PROPHET,0.590,0.591,0.591,0.370,0.370
CNN,0.377,0.381,0.382,0.390,0.378
LSTM,0.470,0.463,0.434,0.358,0.333
LSTM-2,0.450,0.437,0.437,0.371,0.363


In [98]:
nrmse_final_results.to_csv('nrmse_results.csv')

In [91]:
# lets draw a line plot

line_plot = go.Figure()
line_plot.add_trace(go.Scatter(x = nrmse_final_results.index, y =nrmse_final_results['Grid'], name = "Using grid level time series"))
line_plot.add_trace(go.Scatter(x = nrmse_final_results.index, y =nrmse_final_results['TL - Aggregated'], name = "Using transmission Zone level time series"))
line_plot.add_trace(go.Scatter(x = nrmse_final_results.index, y =nrmse_final_results['SUB - Aggregated'], name = "Using substation level time series"))
line_plot.add_trace(go.Scatter(x = nrmse_final_results.index, y =nrmse_final_results['PC - Aggregated'], name = "Using postcode level time series"))
line_plot.add_trace(go.Scatter(x = nrmse_final_results.index, y =nrmse_final_results['Site - Aggregated'], name = "Using site level time series"))


In [92]:
line_plot = go.Figure()
line_plot.add_trace(go.Scatter(x = mase_final_results.index, y =mase_final_results['Grid'], name = "Using grid level time series"))
line_plot.add_trace(go.Scatter(x = mase_final_results.index, y =mase_final_results['TL - Aggregated'], name = "Using transmission Zone level time series"))
line_plot.add_trace(go.Scatter(x = mase_final_results.index, y =mase_final_results['SUB - Aggregated'], name = "Using substation level time series"))
line_plot.add_trace(go.Scatter(x = mase_final_results.index, y =mase_final_results['PC - Aggregated'], name = "Using postcode level time series"))
line_plot.add_trace(go.Scatter(x = nrmse_final_results.index, y =mase_final_results['Site - Aggregated'], name = "Using site level time series"))


In [72]:
# create a dataframe
grid_results = grid_error
grid_results.extend(grid_error_tcn)
grid_results.extend(grid_error_arima)
grid_results.extend(grid_error_tbats)
grid_results.extend(grid_error_prophet)
grid_results.extend([grid_error_naive[0]])
len(grid_results)

26

In [73]:
column_names = ['LSTM - Grid', 'LSTM - TL aggregated', 'LSTM - SUB aggregated', 'LSTM - PC aggregated', 
                'LSTM - Site aggregated', 'CNN - Grid', 'CNN - TL aggregated', 'CNN - SUB aggregated', 
                'CNN - PC aggregated', 'CNN - Site aggregated', 'AutoARIMA - Grid', 'AutoARIMA - TL aggregated', 
                'AutoARIMA - SUB aggregated', 
                'AutoARIMA - PC aggregated', 'AutoARIMA - Site aggregated', 'TBATS - Grid', 
                'TBATS - TL aggregated', 'TBATS - SUB aggregated', 
                'TBATS - PC aggregated', 'TBATS - Site aggregated','Prophet - Grid', 
                'Prophet - TL aggregated', 'Prophet - SUB aggregated', 
                'Prophet - PC aggregated', 'Prophet - Site aggregated','Seasonal Naive']

grid_level_fc_accuracy = pd.DataFrame(grid_results, index = column_names, columns = ['Grid Forecasts mean MASE'])

In [74]:
def highlight_min(s):
    is_min = grid_level_fc_accuracy.isin(grid_level_fc_accuracy.nsmallest(3, 'Grid Forecasts mean MASE')).values[:,0]
    return ['background-color: lightgreen' if v else '' for v in is_min]

In [75]:
grid_level_fc_accuracy.style.apply(highlight_min)

,Grid Forecasts mean MASE
LSTM - Grid,1.468234
LSTM - TL aggregated,1.416184
LSTM - SUB aggregated,1.330124
LSTM - PC aggregated,1.123227
LSTM - Site aggregated,1.013093
CNN - Grid,1.150049
CNN - TL aggregated,1.169436
CNN - SUB aggregated,1.171721
CNN - PC aggregated,1.198341
CNN - Site aggregated,1.169582


In [76]:
final_result_csv = grid_level_fc_accuracy.round(3)

In [77]:
final_result_csv.to_csv('final_results.csv')

In [80]:
# name = ['LSTM - Grid', 'LSTM - TL aggregated', 'LSTM - SUB aggregated', 'LSTM - PC aggregated', 
#                 'LSTM - Site aggregated', 'CNN - Grid', 'CNN - TL aggregated', 'CNN - SUB aggregated', 
#                 'CNN - PC aggregated', 'CNN - Site aggregated', 'Auto ARIMA - Grid']

fig = go.Figure()
count = 0
for col in grid_error_dist.keys():
    box_data = grid_error_dist[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean=True))
    count = count+1
    
for col in grid_error_dist_tcn.keys():
    box_data = grid_error_dist_tcn[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    
for col in grid_error_dist_arima.keys():
    box_data = grid_error_dist_arima[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    
for col in grid_error_dist_tbats.keys():
    box_data = grid_error_dist_tbats[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    
for col in grid_error_dist_prophet.keys():
    box_data = grid_error_dist_prophet[col]
    fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
    count = count+1
    

box_data = grid_error_dist_naive['grid']
fig.add_trace(go.Box(y=box_data, name = column_names[count], boxmean = True))
count = count+1
    

fig.update_xaxes(title = '')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "", showlegend = False, font = dict(size=10))

In [78]:
x_vals = ['Grid', 'TL Aggregated', 'SUB Aggregated', 
          'PC Aggregated', 'Site Aggregated']
fig = go.Figure(data=[
    go.Bar(name='LSTM', x=x_vals, y=grid_error),
    go.Bar(name='Convolutions', x=x_vals, y=grid_error_tcn)
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.update_layout(title='MASE at Grid Level')
fig.show()

In [35]:
fig = go.Figure()
for col in grid_error_dist.keys():
    box_data = grid_error_dist[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'BU strategy - ts hierarchy level')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Grid level forecasts - Model LSTM", showlegend = False)

In [34]:
fig = go.Figure()
for col in grid_error_dist_tcn.keys():
    box_data = grid_error_dist_tcn[col]
    fig.add_trace(go.Box(y=box_data, name = col))
fig.update_xaxes(title = 'BU strategy - ts hierarchy level')
fig.update_yaxes(title = 'MASE distribution')
fig.update_layout(title = "Grid level forecasts - Model Dilated Convolutions", showlegend=False)